In [1]:
from ctapipe.io import event_source
from ctapipe.calib import CameraCalibrator
from ctapipe.utils import get_dataset
from ctapipe.image import tailcuts_clean
from ctapipe.reco import HillasReconstructor
from ctapipe.image import hillas_parameters
from ctapipe.reco.HillasReconstructor import TooFewTelescopesException
import hipecta.memory
import hipecta.calib
import astropy.units as u
import numpy as np
import copy

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
filename = get_dataset("gamma_test.simtel.gz")

In [4]:
events = [copy.deepcopy(event) for event in event_source(filename)]

In [5]:
cal = CameraCalibrator(None, None, r1_product='HESSIOR1Calibrator', extractor_product='NeighbourPeakIntegrator')

In [6]:
hillas_reco = HillasReconstructor()

In [7]:
class HillasNotFinite(Exception):
    """
    Error to be raised when hillas parameters are not finite
    """
    pass

In [21]:
def reconstruction(event, picture_thresh=10, boundary_thresh=5):
    
    cal.calibrate(event)
    
    features = {}
    hillas_dict = {}
    pointing_azimuth = {}
    pointing_altitude = {}
    for telescope_id, dl1 in event.dl1.tel.items():

        camera = event.inst.subarray.tels[telescope_id].camera
        mask = tailcuts_clean(camera,
                              dl1.image[0], 
                              boundary_thresh=boundary_thresh, 
                              picture_thresh=picture_thresh)
        
        telescope_type_name = event.inst.subarray.tels[telescope_id].optics.tel_type
        image = dl1.image[0]
        image[~mask] = 0

        if image.sum() > 0:
            try:
                h = hillas_parameters(
                    camera,
                    image
                )

                if not all(map(np.isfinite, h.values())):
                    raise HillasNotFinite("bad Hillas parameters")
                    
                hillas_dict[telescope_id] = h
                pointing_azimuth[telescope_id] = event.mc.tel[telescope_id].azimuth_raw * u.rad
                pointing_altitude[telescope_id] = event.mc.tel[telescope_id].altitude_raw * u.rad
                # pointing_altitude[telescope_id] = ((np.pi/2) - event.mc.tel[telescope_id].altitude_raw )* u.rad # this is weird to say the least. 
    
            except HillasNotFinite:
                pass
                
        else:
            print("image sum < 0", image.sum())

    if len(hillas_dict) < 2:
        print("mono")
        # raise TooFewTelescopesException()
        reco = None
    else:
        reco = hillas_reco.predict(hillas_dict, event.inst, pointing_azimuth, pointing_altitude)
   
    return reco


In [22]:
def process_events(filename):
    events = [copy.deepcopy(event) for event in event_source(filename)]
    print(len(events))
    reco = []
    for i, event in enumerate(events):
        print(i)
        reco.append(reconstruction(event))
    return reco

In [23]:
reco = process_events(filename);

9
0
image sum < 0 0.0
image sum < 0 0.0
mono
1
image sum < 0 0.0
2
image sum < 0 0.0
3
4
5
6
7
8
image sum < 0 0.0
image sum < 0 0.0
image sum < 0 0.0


In [27]:
reco[4].alt

<Latitude 0.01714089 rad>

In [29]:
for r, event in zip(reco, events):
    print(event.r0.event_id)
    print(event.mc.alt, event.mc.az)
    print(r)
    print('-----')

408
1.22173rad 6.28319rad
None
-----
409
1.22173rad 6.28319rad
{'alt': <Latitude nan rad>,
 'alt_uncert': <Quantity nan rad>,
 'average_size': 207.62513161665413,
 'az': <Longitude nan rad>,
 'az_uncert': nan,
 'core_uncert': [nan, nan],
 'core_x': nan,
 'core_y': nan,
 'goodness_of_fit': nan,
 'h_max': <Quantity 10000. m>,
 'h_max_uncert': nan,
 'is_valid': True,
 'tel_ids': [21, 119, 24, 26, 11, 63]}
-----
803
1.22173rad 6.28319rad
{'alt': <Latitude nan rad>,
 'alt_uncert': <Quantity nan rad>,
 'average_size': 43.8573701448595,
 'az': <Longitude nan rad>,
 'az_uncert': nan,
 'core_uncert': [nan, nan],
 'core_x': nan,
 'core_y': nan,
 'goodness_of_fit': nan,
 'h_max': <Quantity 10000. m>,
 'h_max_uncert': nan,
 'is_valid': True,
 'tel_ids': [17, 124]}
-----
4907
1.22173rad 6.28319rad
{'alt': <Latitude 0.0060795 rad>,
 'alt_uncert': <Quantity 0.59974088 rad>,
 'average_size': 52.10743741736368,
 'az': <Longitude 1.21962609 rad>,
 'az_uncert': nan,
 'core_uncert': <Quantity 0.02883571 m